In [1]:
#| default_exp devcontainer.devcontainer

# devcontainer

https://containers.dev/ - An open specification for enriching containers with development specific content and settings.

In [2]:
#| export
import os
from pathlib import Path

from jinja2 import Environment

from sparrow_patterns.utils import get_source_directory

In [3]:
#| hide
import tempfile

In [4]:
#| export
def generate_devcontainer(
    project_name: str, # The slug for the project. Should be the same as the GitHub repo.
    package: bool = False, # Whether this is for a Python package
    gpu: bool = False, # Whether to make the GPU available
    project_directory: str = ".", # Where to create the .devcontainer directory. Defaults to working directory.
    template_directory: str = "." # Where the template directory is located. Defaults to working directory.
) -> None:
    """
    Write a .devcontainer directory for the project.

    Parameters
    ----------
    project_name
        The slug for the project. Should be the same as the GitHub repo.
    package
        Whether this is for a Python package
    gpu
        Whether to make the GPU available
    project_directory
        Where to create the .devcontainer folder. Defaults to working directory.
    template_directory
        Where the template directory is located. Defaults to working directory.
    """
    env = Environment(autoescape=True)
    source_directory = get_source_directory(project_name)
    template_directory = Path(template_directory) / "templates"
    output_directory = Path(project_directory) / ".devcontainer"
    output_directory.mkdir(exist_ok=True)
    template_variables = dict(
        project_name=project_name,
        source_directory=source_directory,
        package=package,
        gpu=gpu,
    )
    for fname in os.listdir(template_directory):
        if package and fname in (".env.example", ".gitignore"):
            continue
        output_path = output_directory / fname
        with open(template_directory / fname) as f:
            template = env.from_string(f.read())
        with open(output_path, "w") as f:
            f.write(template.render(**template_variables))

In [5]:
#| hide
with tempfile.TemporaryDirectory() as project_directory:
    generate_devcontainer(
        project_name="foo-bar",
        package=True,
        project_directory=project_directory,
        template_directory='../sparrow_patterns/devcontainer'
    )
    source_directory = "foo_bar"
    env_example_path = os.path.join(project_directory, ".devcontainer/.env.example")
    gitignore_path = os.path.join(project_directory, ".devcontainer/.gitignore")
    assert os.path.exists(env_example_path) == False
    assert os.path.exists(gitignore_path) == False
    config_path = os.path.join(project_directory, ".devcontainer/devcontainer.json")
    with open(config_path) as f:
        assert source_directory in f.read()

In [6]:
#| hide
with tempfile.TemporaryDirectory() as project_directory:
    generate_devcontainer(
        project_name="hello-world",
        project_directory=project_directory,
        template_directory='../sparrow_patterns/devcontainer'
    )
    env_example_path = os.path.join(project_directory, ".devcontainer/.env.example")
    with open(env_example_path) as f:
        assert "AWS_PROFILE=sparrow" in f.read().splitlines()
    config_path = os.path.join(project_directory, ".devcontainer/devcontainer.json")
    with open(config_path) as f:
        assert "source=${localEnv:HOME}/.darwin,target=/root/.darwin" in f.read()

In [7]:
#| hide
import nbdev; nbdev.nbdev_export()